### Converting an Allen-Style Taxonomy Spreadsheet to CAS Format  

This notebook demonstrates how to convert an Allen-style taxonomy spreadsheet into the CAS (Cell Annotation Schema) format. The process includes the following steps:  

1. **Generate CAS**: Convert the spreadsheet data into the CAS format.
2. **Validate CAS against h5ad**: Compare CAS and AnnData to check if the annotations and the hierarchy are the same.
3. **Populate IDs**: Add corresponding Cell IDs to the CAS from a related Anndata (h5ad) file.  
4. **Merge Data**: Integrate the updated CAS into the `var` field of the Anndata object.  

This workflow uses a subset of the WMBO dataset. Example Allen-style spreadsheet is derived from the `CB Glut (CS20230722_CLAS_29)` subset of the Mouse Cell-type annotations (https://www.nature.com/articles/s41586-023-06812-z#Sec49, Table 7)

This notebook demonstrates how to carry out these steps in Python, but most can also be done via the [cas-tools command-line interface](https://cellannotation.github.io/cas-tools/cli.html).

#### Installing Required Packages

For detailed instructions on setting up and running the notebooks, please refer to the [README.md](https://github.com/cellannotation/cas-tools/blob/main/notebooks/README.md) in the notebooks directory.

#### Example of an Allen-Style Spreadsheet  

To keep things simple, we use the `CB Glut (CS20230722_CLAS_29)` subset from the Mouse Cell-Type Annotations dataset (refer to [Nature article, Supplementary Table 7](https://www.nature.com/articles/s41586-023-06812-z#Sec49)).

In [1]:
import pandas as pd

pd.read_csv("./data/wmb_class_29_annotation.tsv", delimiter='\t')[:4]

,cluster_id,cluster,supertype,subclass,class,neighborhood,anatomical_annotation,notes,CCF_broad.freq,CCF_acronym.freq,...,CTX.subclass_id,CTX.subclass_id.1,CTX.neighborhood_id,CTX.neighborhood_label,CTX.size,taxonomy_id,cell_set_accession.cluster,cell_set_accession.supertype,cell_set_accession.subclass,cell_set_accession.class
0,5197,5197 CB Granule Glut_1,1154 CB Granule Glut_1,314 CB Granule Glut,29 CB Glut,NN-IMN-GC,DCO VCO,NaN,"MY:0.35,CB:0.33,NA:0.26","DCO:0.19,FL:0.11,arb:0.09,VCO:0.09,DN:0.08,PFL...",...,NaN,NaN,NaN,NaN,NaN,CCN202307220,CS20230722_CLUS_5197,CS20230722_SUPT_1154,CS20230722_SUBC_314,CS20230722_CLAS_29
1,5198,5198 CB Granule Glut_1,1154 CB Granule Glut_1,314 CB Granule Glut,29 CB Glut,NN-IMN-GC,DCO VCO,NaN,"CB:0.56,MY:0.24,NA:0.19","FL:0.25,PFL:0.17,DCO:0.13,VCO:0.09,arb:0.08,NA...",...,NaN,NaN,NaN,NaN,NaN,CCN202307220,CS20230722_CLUS_5198,CS20230722_SUPT_1154,CS20230722_SUBC_314,CS20230722_CLAS_29
2,5199,5199 CB Granule Glut_1,1154 CB Granule Glut_1,314 CB Granule Glut,29 CB Glut,NN-IMN-GC,DCO VCO,NaN,"MY:0.4,CB:0.38,NA:0.18","DCO:0.29,DN:0.12,arb:0.09,FL:0.09,PFL:0.06,VCO...",...,NaN,NaN,NaN,NaN,NaN,CCN202307220,CS20230722_CLUS_5199,CS20230722_SUPT_1154,CS20230722_SUBC_314,CS20230722_CLAS_29
3,5200,5200 CB Granule Glut_2,1155 CB Granule Glut_2,314 CB Granule Glut,29 CB Glut,NN-IMN-GC,NOD PFL,NaN,"CB:0.77,NA:0.22","PFL:0.17,NOD:0.17,FL:0.13,NA:0.11,arb:0.1,UVU:...",...,NaN,NaN,NaN,NaN,NaN,CCN202307220,CS20230722_CLUS_5200,CS20230722_SUPT_1155,CS20230722_SUBC_314,CS20230722_CLAS_29


#### Import Spreadsheet into CAS  

Import assumes that the source spreadsheet will have:

* One row per cluster (where cluster is the most granular cell set)
* A column containing cluster_ids
* A spreadsheet representation of hierarchy with columns for each level in the hierarchy.
* A set of columns specific to the sheet (author categories)

To import an Allen-style spreadsheet into the CAS, we need a simple mapping file. This includes some general metadata and specification of heirarchy fields.  Rank refers to the level of a heirarchy field - with 0 being the most granular. The mapping file must specify cluster_id and cluster_name require specific field types.  All other hierarchy fields are typed as cell set.  Here is a configuration for our example sheet:

In [ ]:
!less ./data/wmb_ingestion_config.yaml

taxonomy_id: CS20230722
brain_region_names:
  - Whole Brain
species_names:
  - Mouse
author_name: Hongkui Zeng
title: Whole Mouse Brain taxonomy

fields:
  - column_name: cluster_id
    column_type: cluster_id
    rank: 0

  - column_name: cluster
    column_type: cluster_name
    rank: 0

  - column_name: supertype
    column_type: cell_set
    rank: 1

  - column_name: subclass
    column_type: cell_set
./data/wmb_ingestion_config.yaml

In [13]:
# Import to CAS, save JSON file and display resulting JSON:

import json
from cas.ingest.ingest_user_table import ingest_data, ingest_user_data

cas_data = ingest_user_data("./data/wmb_class_29_annotation.tsv", "./data/wmb_ingestion_config.yaml", True)
wmb_class_29_cas = cas_data.as_dictionary()
with open('wmb_class_29_cas.json', 'w') as f:
    f.write(json.dumps(wmb_class_29_cas))
            
print(json.dumps(wmb_class_29_cas, indent=2)[:2000])

{
  "author_name": "Hongkui Zeng",
  "annotations": [
    {
      "labelset": "class",
      "cell_label": "29 CB Glut",
      "cell_set_accession": "5206"
    },
    {
      "labelset": "subclass",
      "cell_label": "314 CB Granule Glut",
      "cell_set_accession": "5207",
      "parent_cell_set_accession": "5206"
    },
    {
      "labelset": "supertype",
      "cell_label": "1154 CB Granule Glut_1",
      "cell_set_accession": "5208",
      "parent_cell_set_accession": "5207"
    },
    {
      "labelset": "cluster",
      "cell_label": "5197 CB Granule Glut_1",
      "cell_set_accession": "5197",
      "parent_cell_set_accession": "5208",
      "author_annotation_fields": {
        "neighborhood": "NN-IMN-GC",
        "anatomical_annotation": "DCO VCO",
        "CCF_broad.freq": "MY:0.35,CB:0.33,NA:0.26",
        "CCF_acronym.freq": "DCO:0.19,FL:0.11,arb:0.09,VCO:0.09,DN:0.08,PFL:0.07,MY:0.07,mcp:0.06,CUL4, 5:0.04",
        "v3.size": "14544",
        "v2.size": "0",
        "m

In [14]:
# Rather than viewing the JSON we can use reporting tools from cas-tools to view extended annotations as dataframes:

from cas import reports
reports.get_all_annotations(wmb_class_29_cas)

,labelset,cell_label,cell_set_accession,parent_cell_set_accession,author_annotation_fields.neighborhood,author_annotation_fields.anatomical_annotation,author_annotation_fields.CCF_broad.freq,author_annotation_fields.CCF_acronym.freq,author_annotation_fields.v3.size,author_annotation_fields.v2.size,...,author_annotation_fields.cluster.markers.combo,author_annotation_fields.merfish.markers.combo,author_annotation_fields.cluster.TF.markers.combo,author_annotation_fields.cluster.markers.combo (within subclass),author_annotation_fields.taxonomy_id,author_annotation_fields.cell_set_accession.cluster,author_annotation_fields.cell_set_accession.supertype,author_annotation_fields.cell_set_accession.subclass,author_annotation_fields.cell_set_accession.class,author_annotation_fields.np.markers
0,class,29 CB Glut,5206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,subclass,314 CB Granule Glut,5207,5206,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,supertype,1154 CB Granule Glut_1,5208,5207,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cluster,5197 CB Granule Glut_1,5197,5208,NN-IMN-GC,DCO VCO,"MY:0.35,CB:0.33,NA:0.26","DCO:0.19,FL:0.11,arb:0.09,VCO:0.09,DN:0.08,PFL...",14544,0,...,"Gabra6,Lmx1a,Rnf182","Col27a1,Barhl1,St18,Trhde,Spon1,Syt6","Lmx1a,Zic1,St18","Lmx1a,Rnf182",CCN202307220,CS20230722_CLUS_5197,CS20230722_SUPT_1154,CS20230722_SUBC_314,CS20230722_CLAS_29,NaN
4,cluster,5198 CB Granule Glut_1,5198,5208,NN-IMN-GC,DCO VCO,"CB:0.56,MY:0.24,NA:0.19","FL:0.25,PFL:0.17,DCO:0.13,VCO:0.09,arb:0.08,NA...",2196,0,...,"Gabra6,Cntn5","Svep1,Slc17a7,Chrm2","Pax6,Neurod2,Etv1,Bcl11b",Cntn5,CCN202307220,CS20230722_CLUS_5198,CS20230722_SUPT_1154,CS20230722_SUBC_314,CS20230722_CLAS_29,NaN
5,cluster,5199 CB Granule Glut_1,5199,5208,NN-IMN-GC,DCO VCO,"MY:0.4,CB:0.38,NA:0.18","DCO:0.29,DN:0.12,arb:0.09,FL:0.09,PFL:0.06,VCO...",909,0,...,"Cbln3,Tmem132d","Eomes,Col27a1,Calb2","Eomes,Lmx1a,Nr2f2,Lin28b",Rgs6,CCN202307220,CS20230722_CLUS_5199,CS20230722_SUPT_1154,CS20230722_SUBC_314,CS20230722_CLAS_29,NaN
6,supertype,1155 CB Granule Glut_2,5209,5207,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,cluster,5200 CB Granule Glut_2,5200,5209,NN-IMN-GC,NOD PFL,"CB:0.77,NA:0.22","PFL:0.17,NOD:0.17,FL:0.13,NA:0.11,arb:0.1,UVU:...",220,0,...,"Gabra6,Gap43,Il1rap","Eomes,Svep1,Ntng1,Medag","Eomes,St18,En2,Nr4a2","Gap43,Kcnq5",CCN202307220,CS20230722_CLUS_5200,CS20230722_SUPT_1155,CS20230722_SUBC_314,CS20230722_CLAS_29,NaN
8,cluster,5201 CB Granule Glut_2,5201,5209,NN-IMN-GC,CBX,"CB:0.74,NA:0.26","NA:0.17,CUL4, 5:0.14,SIM:0.11,ANcr1:0.09,arb:0...",115909,0,...,"Gabra6,Daam2,Calb2","Svep1,Slc17a7,Pappa,Barhl2","Pax6,En2,Etv1,Barhl2,Uncx,Maf",Clstn2,CCN202307220,CS20230722_CLUS_5201,CS20230722_SUPT_1155,CS20230722_SUBC_314,CS20230722_CLAS_29,NaN
9,subclass,315 DCO UBC Glut,5210,5206,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Retrieve AnnData File for `CB Glut`

Assumed starting point - an AnnData file with annotations corresponding to the hierarchy in the spreadsheet.

Download the AnnData file corresponding to `CB Glut (CS20230722_CLAS_29)`. The original [WMB-10Xv2](https://allen-brain-cell-atlas.s3.us-west-2.amazonaws.com/index.html#expression_matrices/WMB-10Xv2/20230630/) and [WMB-10Xv3](https://allen-brain-cell-atlas.s3.us-west-2.amazonaws.com/index.html#expression_matrices/WMB-10Xv3/20230630/) AnnData files were provided as separate files per dissection.  

These files were merged and then split into 34 top-level classes. The resulting AnnData files for each class can be accessed at the following links:  
- [Class 01](http://cellular-semantics.cog.sanger.ac.uk/public/merged_CS20230722_CLAS_01.h5ad)  
- [Class 02](http://cellular-semantics.cog.sanger.ac.uk/public/merged_CS20230722_CLAS_02.h5ad)  
- ...  
- [Class 34](http://cellular-semantics.cog.sanger.ac.uk/public/merged_CS20230722_CLAS_34.h5ad)  

The file specific to `CB Glut` is included in this collection. For this tutorial, we will use a version that lacks embedded CAS.

In [5]:
!wget -N http://cellular-semantics.cog.sanger.ac.uk/public/CS20230722_CLAS_29.h5ad

--2025-03-06 15:10:38--  http://cellular-semantics.cog.sanger.ac.uk/public/CS20230722_CLAS_29.h5ad
Resolving cellular-semantics.cog.sanger.ac.uk (cellular-semantics.cog.sanger.ac.uk)... 193.62.203.61, 193.62.203.63, 193.62.203.62
Connecting to cellular-semantics.cog.sanger.ac.uk (cellular-semantics.cog.sanger.ac.uk)|193.62.203.61|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cellular-semantics.cog.sanger.ac.uk/public/CS20230722_CLAS_29.h5ad [following]
--2025-03-06 15:10:38--  https://cellular-semantics.cog.sanger.ac.uk/public/CS20230722_CLAS_29.h5ad
Connecting to cellular-semantics.cog.sanger.ac.uk (cellular-semantics.cog.sanger.ac.uk)|193.62.203.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1113273953 (1.0G) [binary/octet-stream]
Saving to: ‘CS20230722_CLAS_29.h5ad’

CS20230722_CLAS_29. 100%[===================>]   1.04G  7.89MB/s    in 2m 10s  

2025-03-06 15:12:48 (8.19 MB/s) - ‘CS20230722_CLAS_29.h5

In [16]:
import anndata as ad
C29_ad =  ad.read_h5ad("./CS20230722_CLAS_29.h5ad", backed="r")
C29_ad.obs[:3]

,cell_barcode,library_label,tissue,tissue_ontology_term_id,class,subclass,supertype,cluster,organism,disease,organism_ontology_term_id,disease_ontology_term_id
cell_label,,,,,,,,,,,,
AAACCCAAGAACAAGG-472_A05,AAACCCAAGAACAAGG,L8TX_201217_01_G07,Cerebellum,UBERON:0002037,29 CB Glut,314 CB Granule Glut,1155 CB Granule Glut_2,5201 CB Granule Glut_2,Mus musculus,normal,NCBITaxon:10090,PATO:0000461
AAACCCAAGAATCCCT-473_A06,AAACCCAAGAATCCCT,L8TX_201217_01_A08,Cerebellum,UBERON:0002037,29 CB Glut,314 CB Granule Glut,1155 CB Granule Glut_2,5201 CB Granule Glut_2,Mus musculus,normal,NCBITaxon:10090,PATO:0000461
AAACCCAAGACTACCT-225_A01,AAACCCAAGACTACCT,L8TX_200227_01_F10,Medulla,UBERON:0001896,29 CB Glut,314 CB Granule Glut,1154 CB Granule Glut_1,5197 CB Granule Glut_1,Mus musculus,normal,NCBITaxon:10090,PATO:0000461


In [17]:
C29_ad.file.close()

#### Populate CAS with Cell IDs  

Add Cell IDs to the CAS using data from the AnnData file.  

This first checks that the heirarchy of nested cell sets in the AnnData file matches the heirarchy in the CAS file (as derived from the Allen spreadsheet). If two cell set are identical but one has a higher rank then the other, this is also treated as valid heirarchy.

If the heirarchies match, Cell_IDs are added to the CAS representation.

By default, cell IDs are only added at the most granular level (clusters), as Cell IDs for other levels in the hierarchy can be derived.

In [10]:
add_cell_ids?

Signature:
add_cell_ids(
    cas: dict,
    ad: Union[pandas.core.frame.DataFrame, cap_anndata.backed_df.CapAnnDataDF],
    labelsets: list = None,
)
Docstring:
Add/update CellIDs to CAS from matching AnnData file.

Parameters:
    cas: CAS JSON object
    ad: anndata object
    labelsets: List of labelsets to update with IDs from AnnData. If value is null, rank '0' labelset is used. The
    labelsets should be provided in order, starting from rank 0 (leaf nodes) and ascending to higher ranks.
File:      ~/Documents/GitHub/cas-tools/venv/lib/python3.9/site-packages/cas/populate_cell_ids.py
Type:      function

In [20]:
!cas populate_cells --json ./wmb_class_29_cas.json --anndata ./CS20230722_CLAS_29.h5ad

INFO:root:All labelsets exist in obs.
INFO:root:All labelset members exist in the corresponding obs columns.


In [24]:
with open('wmb_class_29_cas.json', 'r') as f:
    j = json.loads(f.read())
    print(json.dumps(j, indent=2)[:2000])

{
  "author_name": "Hongkui Zeng",
  "annotations": [
    {
      "labelset": "class",
      "cell_label": "29 CB Glut",
      "cell_set_accession": "5206"
    },
    {
      "labelset": "subclass",
      "cell_label": "314 CB Granule Glut",
      "cell_set_accession": "5207",
      "parent_cell_set_accession": "5206"
    },
    {
      "labelset": "supertype",
      "cell_label": "1154 CB Granule Glut_1",
      "cell_set_accession": "5208",
      "parent_cell_set_accession": "5207"
    },
    {
      "labelset": "cluster",
      "cell_label": "5197 CB Granule Glut_1",
      "cell_set_accession": "5197",
      "parent_cell_set_accession": "5208",
      "author_annotation_fields": {
        "neighborhood": "NN-IMN-GC",
        "anatomical_annotation": "DCO VCO",
        "CCF_broad.freq": "MY:0.35,CB:0.33,NA:0.26",
        "CCF_acronym.freq": "DCO:0.19,FL:0.11,arb:0.09,VCO:0.09,DN:0.08,PFL:0.07,MY:0.07,mcp:0.06,CUL4, 5:0.04",
        "v3.size": "14544",
        "v2.size": "0",
        "m

In [21]:
#from cas.populate_cell_ids import add_cell_ids

#labelset_names = [labelset.name for labelset in sorted(cas_data.labelsets, key=lambda x: x.rank)]
#cas = add_cell_ids(wmb_class_29_cas, anndata, labelset_names)

#print(json.dumps(cas["annotations"][5], indent=2)[:1500])

#### Merge CAS to Anndata

Adds CAS to uns of the AnnData file.  Note that further checks are carried out here to assess whether cell set membership matches between the CAS file and the AnnData file.  In this case they do match, but this also protects against files being out of sync.

In [25]:
! cas merge --json ./wmb_class_29_cas.json --anndata ./CS20230722_CLAS_29.h5ad --output CS20230722_CLAS_29_merged.h5ad

In [ ]:
from cas.anndata_conversion import merge_cas_object
import anndata as ad

anndata.file.close()
merge_cas_object(wmb_class_29_cas, "./merged_CS20230722_CLAS_29.h5ad", True, "./merged_CS20230722_CLAS_29_v2.h5ad")

In [27]:
from cas.file_utils import read_cas_from_anndata

cas_read = read_cas_from_anndata("./CS20230722_CLAS_29_merged.h5ad")
cas_read.get_all_annotations().head(5)

,labelset,cell_label,cell_set_accession,cell_fullname,cell_ontology_term_id,cell_ontology_term,rationale,rationale_dois,marker_gene_evidence,synonyms,...,author_annotation_fields.cluster.markers.combo,author_annotation_fields.merfish.markers.combo,author_annotation_fields.cluster.TF.markers.combo,author_annotation_fields.cluster.markers.combo (within subclass),author_annotation_fields.taxonomy_id,author_annotation_fields.cell_set_accession.cluster,author_annotation_fields.cell_set_accession.supertype,author_annotation_fields.cell_set_accession.subclass,author_annotation_fields.cell_set_accession.class,author_annotation_fields.np.markers
0,class,29 CB Glut,5206,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,subclass,314 CB Granule Glut,5207,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,supertype,1154 CB Granule Glut_1,5208,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cluster,5197 CB Granule Glut_1,5197,None,None,None,None,None,None,None,...,"Gabra6,Lmx1a,Rnf182","Col27a1,Barhl1,St18,Trhde,Spon1,Syt6","Lmx1a,Zic1,St18","Lmx1a,Rnf182",CCN202307220,CS20230722_CLUS_5197,CS20230722_SUPT_1154,CS20230722_SUBC_314,CS20230722_CLAS_29,NaN
4,cluster,5198 CB Granule Glut_1,5198,None,None,None,None,None,None,None,...,"Gabra6,Cntn5","Svep1,Slc17a7,Chrm2","Pax6,Neurod2,Etv1,Bcl11b",Cntn5,CCN202307220,CS20230722_CLUS_5198,CS20230722_SUPT_1154,CS20230722_SUBC_314,CS20230722_CLAS_29,NaN


In [30]:

!cas export2CAP --anndata ./CS20230722_CLAS_29_merged.h5ad --output ./CS20230722_CLAS_29_CAP.h5ad 

INFO:root:All labelsets exist in obs.
INFO:root:All labelset members exist in the corresponding obs columns.


# CAP requires that annotations be

In [38]:
import anndata as ad
C29_ad =  ad.read_h5ad("./CS20230722_CLAS_29_CAP.h5ad", backed="r")
print(C29_ad.obs.columns)
print(C29_ad.obs[:2])
C29_ad.file.close()

Index(['cell_barcode', 'library_label', 'tissue', 'tissue_ontology_term_id',
       'organism', 'disease', 'organism_ontology_term_id',
       'disease_ontology_term_id', 'class', 'class--cell_set_accession',
       'class--author_annotation_fields', 'subclass',
       'subclass--cell_set_accession', 'subclass--parent_cell_set_accession',
       'subclass--author_annotation_fields', 'supertype',
       'supertype--cell_set_accession', 'supertype--parent_cell_set_accession',
       'supertype--author_annotation_fields', 'cluster',
       'cluster--cell_set_accession', 'cluster--parent_cell_set_accession',
       'cluster--author_annotation_fields'],
      dtype='object')
                              cell_barcode       library_label      tissue  \
cell_label                                                                   
AAACCCAAGAACAAGG-472_A05  AAACCCAAGAACAAGG  L8TX_201217_01_G07  Cerebellum   
AAACCCAAGAATCCCT-473_A06  AAACCCAAGAATCCCT  L8TX_201217_01_A08  Cerebellum   

        